In [1]:
## FMP is being currently used to source historical price data - it is exporting historical data to individual CSV files for each symbol 
## In theory - should be possible to parse the CSV files for historical data, and append missing data from FMP via IEX and make more 
## complete data frames. FMP has an issue where it can't source fundamentals data (market cap, outstandingshares) for ETFs. IEX can. 
## Need to come up with code to fill in the gaps from FMP with IEX data. Unlike FMP, IEX has limited API calls, so should attempt to 
## source FMP first and then call IEX to fill in gaps 

## Rewrite of code to be more clean, no test code, and to work more on applying more fields of data to "Fundamentals field"

In [2]:
import pandas as pd

import glob

from pathlib import Path
import csv

import os
import requests
import json

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
## IEX Constants
iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

## Redundant Assignment but improves Readability throughout code 
real_token = iex_api_key
test_token = iex_test_api_key

base_url_iex = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'

## IEX Status Test 
test_resp = requests.get(base_url_iex + 'status')
test_resp

<Response [200]>

In [4]:
token_status = test_token ## Set to either real token or test token for IEX

In [5]:
## Load IEX to get ETF statistics 
def get_IEX_statistics(stock_ticker, token=token_status):
    if token == test_token:
        resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/stats/?token='+test_token)
        data_json = resp_data.json()
    elif token == real_token:
        resp_data = requests.get(base_url_iex+'stock/'+stock_ticker+'/stats/?token='+real_token)
        data_json = resp_data.json()
        
    return data_json

In [6]:
## Use pickle module to import and export and save files
import pickle
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [7]:
## Import files using Pickle
imported_data_dict = {}
imported_key_list = []
## Need to set length manually - check files for max length based off file name 
def import_data(loop_length):
    x=0
    y=49
    increment=50
    
    while x < loop_length:
        str_symbol1 = str(x)
        str_symbol2 = str(y)

        pkl_path = Path('../FilesExportCompleteFMP/data_complete_'+str_symbol1+'_'+str_symbol2+'.pkl')
        data_obj = load_obj(pkl_path)
        data_obj_key_list = []
        for key in data_obj.keys():
            data_obj_key_list.append(key)  
        for symbol in data_obj_key_list:
            data = data_obj[symbol]
            imported_data_dict[symbol] = data

        x += increment
        y += increment
        if y > loop_length: y = loop_length
        
    for key in imported_data_dict.keys():
        imported_key_list.append(key) 

In [8]:
total_length = 21782
test_length = 1500
import_data(total_length)

In [9]:
num_files = (total_length-32)/50
num_files

435.0

In [10]:
length_ = len(imported_data_dict)
length_

3573

In [11]:
# ## Code to shorten imported_key_list for testing purposes 
# len(imported_key_list)
# new_key_list = imported_key_list[0:250]
# len(new_key_list)
# imported_key_list = new_key_list
# short_length = len(imported_key_list)
# print(short_length)

In [12]:
## How to find index value of specific symbol
## Useful for testing 
index_value = 0
length_val = len(imported_key_list)
for i in imported_key_list:
    if i == 'ARKQ':
        print(index_value,i)
        break
    if index_value == (length_val-1): print("NO")
    index_value +=1

241 ARKQ


In [13]:
## Seperate data that has all FMP data, and data that is missing FMP data 
full_key_list = []
full_key_data_dict = {}
notfull_key_list = []
notfull_key_data_dict = {}
for i in imported_key_list:
    if imported_data_dict[i]['companyProfile'] != 0:
        full_key_list.append(i)
#         data = imported_data_dict[i]
#         full_key_data_dict[i] = data
    elif imported_data_dict[i]['companyProfile'] == 0:
        notfull_key_list.append(i)
#         data = imported_data_dict[i]
#         notfull_key_data_dict[i] = data  

In [14]:
len(notfull_key_list)

144

In [15]:
iex_token = real_token ## Set to either real token or test token for IEX - 35,545 tokens used
iex_token

'sk_abbc88c6443e49a68210d0009cb92885'

In [16]:
## Run through all imported symbols from ../FilesExportCompleteFMP/..
for i in imported_key_list: 
    iex_etf_stats = get_IEX_statistics(i,token=iex_token)
    #sharesOutstanding = iex_etf_stats['sharesOutstanding']   
    imported_data_dict[i]['Fundamentals']['sharesOutstanding'] = iex_etf_stats['sharesOutstanding']
    imported_data_dict[i]['Fundamentals']['peRatio'] = iex_etf_stats['peRatio']
    imported_data_dict[i]['Fundamentals']['beta'] = iex_etf_stats['beta']
    imported_data_dict[i]['Fundamentals']['week52high'] = iex_etf_stats['week52high']
    imported_data_dict[i]['Fundamentals']['week52low'] = iex_etf_stats['week52low']
    imported_data_dict[i]['Fundamentals']['week52change'] = iex_etf_stats['week52change']
    imported_data_dict[i]['Fundamentals']['avg10Volume'] = iex_etf_stats['avg10Volume']
    imported_data_dict[i]['Fundamentals']['avg30Volume'] = iex_etf_stats['avg30Volume']
    imported_data_dict[i]['Fundamentals']['marketcap_IEX'] = iex_etf_stats['marketcap']    
    
    ## Replace 'marketCap' and put into 'marketcap_FMP', then delete 'marketCap' to seperate sources
    imported_data_dict[i]['Fundamentals']['marketcap_FMP'] = imported_data_dict[i]['Fundamentals']['marketCap']
    del imported_data_dict[i]['Fundamentals']['marketCap']
    del imported_data_dict[i]['Fundamentals']['price_earnings']
    
    imported_data_dict[i]['iex_statistics'] = iex_etf_stats

In [17]:
# for i in full_key_list: 
#     iex_etf_stats = get_IEX_statistics(i)
#     #sharesOutstanding = iex_etf_stats['sharesOutstanding']   
#     full_key_data_dict[i]['Fundamentals']['sharesOutstanding'] = iex_etf_stats['sharesOutstanding']
#     full_key_data_dict[i]['Fundamentals']['peRatio'] = iex_etf_stats['peRatio']
#     full_key_data_dict[i]['Fundamentals']['beta'] = iex_etf_stats['beta']
#     full_key_data_dict[i]['Fundamentals']['week52high'] = iex_etf_stats['week52high']
#     full_key_data_dict[i]['Fundamentals']['week52low'] = iex_etf_stats['week52low']
#     full_key_data_dict[i]['Fundamentals']['week52change'] = iex_etf_stats['week52change']
#     full_key_data_dict[i]['Fundamentals']['avg10Volume'] = iex_etf_stats['avg10Volume']
#     full_key_data_dict[i]['Fundamentals']['avg30Volume'] = iex_etf_stats['avg30Volume']
#     full_key_data_dict[i]['Fundamentals']['marketcap_IEX'] = iex_etf_stats['marketcap']    
    
#     ## Replace 'marketCap' and put into 'marketcap_FMP', then delete 'marketCap' to seperate sources
#     full_key_data_dict[i]['Fundamentals']['marketcap_FMP'] = full_key_data_dict[i]['Fundamentals']['marketCap']
#     del full_key_data_dict[i]['Fundamentals']['marketCap']
#     del full_key_data_dict[i]['Fundamentals']['price_earnings']
    
#     full_key_data_dict[i]['iex_statistics'] = iex_etf_stats
    
# # ## Export equity/etf data 
# # pkl_path = Path('../Resources/full_IEX_FMP_data_dict.pkl')
# # save_obj(full_key_data_dict,pkl_path)

In [18]:
imported_data_dict['XRT']

{'Fundamentals': {'sharesOutstanding': 10700000,
  'floatShares': 0,
  'debt_ratio': 'debt_ratioValue',
  'exchange': 'New York Stock Exchange Arca',
  'final_close_price': 94.29,
  'peRatio': 0,
  'beta': 0.6972426471795934,
  'week52high': 104.31,
  'week52low': 58.7,
  'week52change': 0.6556517112668445,
  'avg10Volume': 3833057,
  'avg30Volume': 3130799,
  'marketcap_IEX': 1065506000,
  'marketcap_FMP': 1204937984},
 'dataFrame':                  open       high        low      close   adjClose     volume  \
 Date                                                                           
 2020-01-02  46.540001  46.540001  45.439999  45.660000  44.978550  3539300.0   
 2020-01-03  45.040001  45.500000  45.009998  45.310001  44.633770  4236000.0   
 2020-01-06  45.099998  45.549999  44.930000  45.470001  44.791386  3350300.0   
 2020-01-07  45.500000  45.680000  45.020000  45.310001  44.633770  2889600.0   
 2020-01-08  45.730000  45.779999  45.189999  45.400002  44.722431  4679700.0

In [19]:
pkl_path_complete= Path('../Resources/combined_FMPIEX_complete_data.pkl')
save_obj(imported_data_dict,pkl_path_complete)